In [1]:
#rishab doing simple CNN on the keras dataset! later used for fish classification

In [3]:
#defining the requried classes!
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten

from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

import os
import glob
from skimage.transform import resize
from skimage.io import imread

Using TensorFlow backend.


In [4]:
dir_name = "/home/rishab/Desktop/kaggle/FishClassification/sample"

In [14]:
#for reproducibility we set seed 
seed = 7
numpy.random.seed(seed)

#we load keras dataset again
#(X_train, y_train), (X_test, y_test) = cifar10.load_data()

import os, glob

folders = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
def load_train(input_path, folders = folders):
    X_train = []
    X_train_id = []
    y_train = []
    
    for fld in folders:
        index = folders.index(fld)
        path = os.path.join(input_path, fld, '*.jpg')
        files = glob.glob(path)
        for fl in files:
            #print("Loading images from "+ fld+ "...")
            flbase = os.path.basename(fl)
            img = resize(imread(fl), (64, 64, 3))
            # rescaling the image
            #img = cv2.resize(img, (32, 32, 3), cv2.INTER_LINEAR).flatten()            
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(fld)

    return X_train, y_train, X_train_id


In [6]:
X_train, y_train, X_train_id = load_train("/home/rishab/Desktop/kaggle/FishClassification/sample", folders) #load fish images

Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
Loading images from ALB...
L

In [7]:
import numpy as np

In [8]:
#np.array(X_train)
#y_train

In [15]:
#we need to normalize the input data from 0-255(pixel values) to a range of 0-1
X_train = np.array(X_train).astype('float')
#X_test = X_test.astype('float32')
X_train = X_train / 255.0
#X_test = X_test / 255.0

In [24]:
#type(y_train)

In [25]:
#output variables are in a vector of 1s and 0s . To model the classification model, we need to get a binary matrix out of this vector.
#this binary matrix will have a width of 10 as there are 10 classes in the data set.
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
#num_classes = len(np.unique(np.array(y_train)))

/home/rishab/anaconda3/lib/python3.5/site-packages/keras/utils/np_utils.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y = np.zeros((len(y), nb_classes))


IndexError: arrays used as indices must be of integer (or boolean) type

In [26]:
import pandas as pd
y_train = np.array(pd.get_dummies(pd.Series(y_train)))

Exception: Data must be 1-dimensional

In [27]:
#structure of the following code is as follows
#1) Convolutional input layer, 32 feature maps with a size of 3×3, a rectifier activation function 
#   and a weight constraint of max norm set to 3.
#2) Dropout set to 20%.
#3) Convolutional layer, 32 feature maps with a size of 3×3, a rectifier activation function 
#   and a weight constraint of max norm set to 3.
#4) Max Pool layer with size 2×2.
#5) Flatten layer.
#6) Fully connected layer with 512 units and a rectifier activation function.
#7) Dropout set to 50%.
#8) Fully connected output layer with 10 units and a softmax activation function.

In [28]:
num_classes = np.shape(y_train)[1]

In [29]:
# Create the model
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(64, 64, 3), border_mode='same', activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))



In [30]:

# Compile model
epochs = 25 #set to a small value for current optimization
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 32, 64, 3)     18464       convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 32, 64, 3)     0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 64, 3)     9248        dropout_1[0][0]                  
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 32, 1)     0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [31]:
#fit the model and get the classification accuracy
model.fit(X_train, y_train,
          validation_split = .3,
          #validation_data=(X_test, y_test), 
          nb_epoch = epochs, batch_size=32)
#scores = model.evaluate(X_test, y_test, verbose=0)
#print("The accuracy is: %.2f%%" % (scores[1]*100))

Train on 2643 samples, validate on 1134 samples
Epoch 1/25
2643/2643 [==============================] - 16s - loss: 1.2904 - acc: 0.6425 - val_loss: 4.4507 - val_acc: 0.0000e+00
Epoch 2/25
2643/2643 [==============================] - 15s - loss: 1.1410 - acc: 0.6504 - val_loss: 5.1452 - val_acc: 0.0000e+00
Epoch 3/25
2643/2643 [==============================] - 15s - loss: 1.1378 - acc: 0.6504 - val_loss: 5.6037 - val_acc: 0.0000e+00
Epoch 4/25
2643/2643 [==============================] - 15s - loss: 1.1287 - acc: 0.6504 - val_loss: 5.4266 - val_acc: 0.0000e+00
Epoch 5/25
2643/2643 [==============================] - 15s - loss: 1.1320 - acc: 0.6504 - val_loss: 5.8565 - val_acc: 0.0000e+00
Epoch 6/25
2643/2643 [==============================] - 15s - loss: 1.1320 - acc: 0.6504 - val_loss: 5.4417 - val_acc: 0.0000e+00
Epoch 7/25
2643/2643 [==============================] - 15s - loss: 1.1310 - acc: 0.6504 - val_loss: 5.7230 - val_acc: 0.0000e+00
Epoch 8/25
2643/2643 [====================

In [32]:
scores = model.evaluate(X_train, y_train, verbose=0)
print("The accuracy is: %.2f%%" % (scores[1]*100))

The accuracy is: 45.51%
